<a href="https://colab.research.google.com/github/yutan0565/colab_git/blob/main/code/MobileNet_V3_Small_%EC%98%88%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##기본 모델 형성

In [1]:
!pip install -q tensorflow-model-optimization

     |████████████████████████████████| 237 kB 16.3 MB/s 


In [29]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import os
import datetime
import time

from tensorflow.keras.applications import VGG16
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
import tensorflow_model_optimization as tfmot

In [4]:
(raw_train_x, raw_train_y), (raw_test_x, raw_test_y) = tf.keras.datasets.cifar10.load_data()

"""
print(raw_train_x.shape)
print(raw_test_x.shape)
print(raw_train_y.shape)
print(raw_test_y.shape)
"""

'\nprint(raw_train_x.shape)\nprint(raw_test_x.shape)\nprint(raw_train_y.shape)\nprint(raw_test_y.shape)\n'

In [5]:
# float을 넣을거면 0~1 사이 값으로 바꿔야함
# integer 형태로 넣어보기
train_x = raw_train_x[:45000].astype(np.float32)/255.0
valid_x = raw_train_x[45000:].astype(np.float32)/255.0
test_x = raw_test_x.astype(np.float32)/255.0


train_y = raw_train_y[:45000]
valid_y = raw_train_y[45000:]
test_y = raw_test_y
labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]


"""
print(train_x.shape)
print(valid_x.shape)
print(test_x.shape)
print(train_y.shape)
print(valid_y.shape)
print(test_y.shape)

def show_sample(i):
  print(raw_train_y[i][0], labels[raw_train_y[i][0]])
  plt.imshow(raw_train_x[i])
  plt.show()

for i in [2, 10, 12, 14]:
  show_sample(i)
"""

'\nprint(train_x.shape)\nprint(valid_x.shape)\nprint(test_x.shape)\nprint(train_y.shape)\nprint(valid_y.shape)\nprint(test_y.shape)\n\ndef show_sample(i):\n  print(raw_train_y[i][0], labels[raw_train_y[i][0]])\n  plt.imshow(raw_train_x[i])\n  plt.show()\n\nfor i in [2, 10, 12, 14]:\n  show_sample(i)\n'

In [6]:
vgg = VGG16(  #weights = 'imagenet',  #그냥 초기화 하는거면, 이거 지우기
                            include_top = False,
                            input_shape=(32,32,3)
                            )

# vgg conv 구조만 사용하고 마지막 FC layer는 다른거 사용

fc_layer = keras.Sequential([
                             layers.Flatten(),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(1024, activation = 'relu'),
                             layers.Dense(10, activation = "sigmoid")
                             ])

model = keras.Sequential([vgg,
                          fc_layer
                          ])
model.summary()
# vgg.summary()
# fc_layer.summary()

58900480/58889256 [==============================] - 94s 2us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
sequential (Sequential)      (None, 10)                2634762   
Total params: 17,349,450
Trainable params: 17,349,450
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Callback 함수 지정 해주기
early_stop = EarlyStopping(patience=30) 
mc = ModelCheckpoint("./best_model/vgg_original_checkpoint", 
                     save_best_only=True,
                     monitor = 'val_loss',
                     verbose = 1,
                     mode = 'min') 
reduce_lr  = ReduceLROnPlateau(monitor = 'val_loss',
                               factor=0.5, 
                               patience=5
                               ) 
# tensorboard 관련 
def make_Tensorboard_dir(dir_name):
  root_logdir = os.path.join(os.curdir, dir_name)
  sub_dir_name = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  return os.path.join(root_logdir, sub_dir_name)

dir_name = "Learning_log"
TB_log_dir = make_Tensorboard_dir(dir_name)
TensorB = TensorBoard(log_dir = TB_log_dir)


#optimizaer  조정 해주기
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

In [8]:
# optimizer, loss 함수를 정의하고,  학습 준비를 한다,  metrics 는 어떤 일이 발생하는지 보여줄 것들
model.compile(optimizer=opt, loss="sparse_categorical_crossentropy", metrics=["accuracy"])



# 한번에 몇개의 데이터 학습하고 가중치 갱신할지 
model.fit(train_x, train_y,
          epochs=100,
          verbose=1,
          batch_size=32,
          #validation_split = 0.1
          validation_data = (valid_x, valid_y),
          callbacks = [early_stop, reduce_lr , mc]
          )

Epoch 1/100
1407/1407 [==============================] - 265s 178ms/step - loss: 0.9211 - accuracy: 0.6831 - val_loss: 0.7242 - val_accuracy: 0.7516

Epoch 00001: val_loss improved from inf to 0.72417, saving model to ./best_model\vgg_original_checkpoint
INFO:tensorflow:Assets written to: ./best_model\vgg_original_checkpoint\assets
Epoch 2/100
1407/1407 [==============================] - 251s 178ms/step - loss: 0.5360 - accuracy: 0.8228 - val_loss: 0.5454 - val_accuracy: 0.8218

Epoch 00002: val_loss improved from 0.72417 to 0.54544, saving model to ./best_model\vgg_original_checkpoint
INFO:tensorflow:Assets written to: ./best_model\vgg_original_checkpoint\assets
Epoch 3/100
1407/1407 [==============================] - 251s 178ms/step - loss: 0.3863 - accuracy: 0.8714 - val_loss: 0.5271 - val_accuracy: 0.8338

Epoch 00003: val_loss improved from 0.54544 to 0.52712, saving model to ./best_model\vgg_original_checkpoint
INFO:tensorflow:Assets written to: ./best_model\vgg_original_checkpoi

In [9]:
loss, acc = model.evaluate(test_x, test_y)
print("loss=",loss)
print("acc=",acc)

y_ = model.predict(test_x)
predicted = np.argmax(y_, axis=1)

print(predicted)

## 75.2
## 67.4

313/313 [==============================] - 16s 50ms/step - loss: 1.5378 - accuracy: 0.87415s - loss: 1.524 - ETA: 4s - l
loss= 1.5377521514892578
acc= 0.8741000294685364
[5 8 8 ... 5 1 7]


In [10]:
model.save("./saved_model/vgg_original.h5")
model.save("./saved_model/vgg_original_checkpoint")

INFO:tensorflow:Assets written to: ./saved_model/vgg_original_checkpoint\assets


##int_8 Quantization 진행

In [11]:
# 구조까지 들어가 있는거
model = tf.keras.models.load_model('./best_model/vgg_original_checkpoint')

In [12]:
# input 에 대해서 변수 설정
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_x).batch(1).take(100):
    yield [input_value]

In [13]:
# int 8 로 quantization 진행하기
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\yutan\AppData\Local\Temp\tmpmq9tz8ps\assets


In [14]:
# 그냥 파일 형태만 tflite로 변환 (float 형태임)
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\yutan\AppData\Local\Temp\tmpy7lkeq1a\assets


INFO:tensorflow:Assets written to: C:\Users\yutan\AppData\Local\Temp\tmpy7lkeq1a\assets


In [15]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [16]:
# 모델 저장 해주는 과정
import pathlib

tflite_models_dir = pathlib.Path("./tflite/vgg_tflite/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"vgg_original_tflite.tflite"
tflite_model_file.write_bytes(tflite_model)
# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"vgg_quantization_tflite.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

17486184

In [23]:
def run_tflite_model(tflite_file, test_image_indices):
  global test_x

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_x[test_image_index]
    test_label = test_y[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()
    print(predictions[i])

  return predictions

In [24]:
def evaluate_model(tflite_file, model_type):
  global test_x
  global test_y

  test_image_indices = range(test_x.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_y.reshape(-1)== predictions) * 100) / len(test_x)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_x)))

In [25]:
evaluate_model(tflite_model_file, model_type="Float")

3
8
1
0
6
6
1
6
3
1
0
9
5
7
9
8
5
7
8
6
9
0
4
9
4
2
4
0
9
6
6
5
4
5
9
6
4
1
9
5
4
6
5
6
0
9
3
8
7
6
9
8
0
5
8
8
7
3
3
2
7
5
6
5
6
2
1
2
3
7
2
6
8
8
0
2
9
3
7
8
8
1
1
7
2
7
4
7
8
9
0
3
8
6
4
6
6
0
0
7
4
5
6
3
1
1
3
6
8
7
4
0
6
2
1
3
0
4
5
7
8
5
1
2
8
0
8
3
5
2
4
1
8
9
1
2
9
7
2
8
6
5
6
3
8
5
6
5
5
2
8
9
6
0
0
5
2
9
3
4
2
1
3
6
0
4
8
4
5
8
9
9
9
8
9
9
3
7
5
0
0
5
2
2
6
8
6
3
3
8
5
8
0
1
7
7
8
8
7
8
5
0
8
7
1
3
0
5
7
9
7
4
5
9
8
0
7
9
8
2
7
6
9
4
3
9
3
4
3
6
5
1
3
8
8
0
4
7
5
5
1
1
8
9
0
3
1
9
2
0
5
3
9
9
4
0
3
0
8
9
8
1
5
7
8
8
2
4
7
0
2
5
6
3
8
3
0
3
4
5
9
0
6
1
0
9
1
0
7
9
1
2
6
1
3
4
6
0
0
6
6
6
5
2
6
1
8
2
1
4
8
6
8
3
4
0
2
7
5
5
3
5
2
3
4
1
7
5
4
6
1
9
3
6
6
9
3
8
0
7
2
6
2
5
8
5
4
6
8
9
9
1
8
2
3
7
5
3
8
0
9
5
8
1
9
4
1
3
2
1
2
7
9
4
2
7
2
7
2
6
6
9
0
9
2
8
7
2
2
5
1
2
6
2
9
6
2
3
0
9
9
8
7
8
8
4
0
1
8
2
7
9
3
6
1
9
0
7
3
7
4
5
8
0
2
9
6
4
6
6
4
3
3
7
7
7
2
5
5
1
1
4
9
9
5
7
5
0
2
0
2
9
7
5
9
4
6
4
4
6
5
6
1
4
3
4
4
3
7
8
4
7
8
0
5
7
6
0
5
4
8
6
8
3
5
9
9
9
4
0
1
0
8
1
1
8
0
2
2
0


8
0
7
3
5
2
5
2
4
5
0
4
8
2
4
0
6
4
5
6
0
8
0
0
4
1
1
6
1
4
2
1
8
2
3
4
4
7
8
8
5
1
8
2
2
3
9
6
6
3
0
7
3
3
2
7
4
5
6
0
2
1
8
1
0
9
3
1
0
5
4
2
3
3
6
7
6
0
0
5
7
4
7
7
0
6
1
7
9
0
9
0
3
8
4
0
8
4
1
0
2
6
4
1
1
2
4
8
4
1
4
7
1
0
0
9
0
4
2
8
8
5
5
0
4
0
8
6
2
5
9
2
9
1
1
5
4
7
5
8
6
4
1
5
7
5


KeyboardInterrupt: 

In [ ]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

In [30]:
def run_tflite_time(tflite_file, test_image_indices):
  global test_x

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_x[test_image_index]
    test_label = test_y[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

        
    start = time.time()
    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]
    end = time.time()
    
    fps = 1 / (end - start)
    predictions[i] = output.argmax()

  return fps

In [31]:
# Change this to test a different image
test_image_index = 1

def test_model_time(tflite_file, test_image_index, model_type):
  global test_y
  fps = run_tflite_time(tflite_file, [test_image_index])
  print(fps)

In [32]:
test_model_time(tflite_model_file, test_image_index, model_type="Float")

39.08294973816135


In [33]:
test_model_time(tflite_model_quant_file, test_image_index, model_type="Quantized")

0.4216510675121595


##Pruning 진행

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    test_x, test_y, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

In [ ]:
class MyDenseLayer(tf.keras.layers.Dense, tfmot.sparsity.keras.PrunableLayer):

  def get_prunable_weights(self):
    # Prune bias also, though that usually harms model accuracy too much.
    return [self.kernel, self.bias]

# Use `prune_low_magnitude` to make the `MyDenseLayer` layer train with pruning.
model_for_pruning = tf.keras.Sequential([
  tfmot.sparsity.keras.prune_low_magnitude(MyDenseLayer(20, input_shape=input_shape)),
  tf.keras.layers.Flatten()
])

model_for_pruning.summary()

In [ ]:
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set. 

num_images = train_x.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs


# Define model for pruning.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.50,
                                                               final_sparsity=0.80,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model_for_pruning.summary()

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(train_x, train_,
                  batch_size=batch_size, epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_x, test_y, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy) 
print('Pruned test accuracy:', model_for_pruning_accuracy)

In [ ]:
%tensorboard --logdir={logdir}

In [ ]:
"""
1.	경량 딥 러닝 기술 배경
  1.	기술 배경
  2.	기술 동향
2.	경량 알고리즘
  1.	모델 구조
    1.	Residual Block (Resnet)
    2.	Densely Connected Convolutional Networks (Densenet)
    3.	Fire module (Squeezenet)
  2.	합성곱 필터 
    1.	Depthwise Convolution (mobilenet)
    2.	Pointwise Convolution
    3.	Depthwise Separable Convolution
  3.	자동 모델 탐색
    1.	NetAdapt
    2.	MNasNet
4.	알고리즘 경량화
  5.	모델 압축
    1.	Weight Pruning
    2.	Quantization & Binarization 
  6.	지식 증류
    1.	전문가 모델 – 숙련가 모델 
  7.	하드웨어 가속화
    1.	TPU
    2.	VPU
    3.	젝슨 TX2
    4.	퀄컴 스냅드래곤
    5.	A12 칩
    6.	엑시노스
    7.	등등 NPU 
  8.	모델 압축 자동 탐색
    1.	Tencent의 PocketFlow
    2.	AutoMC (Automated Model Compression)	
  9.	실험 
    1.	비교 모델
    2.	개발 환경 및 code 구성
    3 .	성능 비교
    4.	Insight
10.	차주 계획


"""